<a href="https://colab.research.google.com/github/Tigeroncode/Intelli-internal-linking/blob/main/Internal_linking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#INTERNAL IMAGE LINKING


## TABLE OF CONTENTS
- REQUIREMENTS
- PHASE -1: DATA COLLECTION
- PHASE -2: AI MODEL DEVELOPMENT
- PHASE -3: LINK RECOMMENDATION ENGINE
- PHASE -4: SITEMAP GENERATION (USING AI)
- PHASE -5: TESTING WITH WIKI API

In [1]:
# ===== GOOGLE COLAB SETUP (CORRECTED) =====
# Install all required packages with correct names
!pip install sentence-transformers chromadb networkx beautifulsoup4
!pip install scrapy pandas numpy scikit-learn spacy
!pip install wikipedia  # Changed from wikipedia-api to wikipedia
!pip install google-api-python-client google-auth-httplib2 google-auth-oauthlib
!pip install requests lxml fake-useragent
!python -m spacy download en_core_web_sm

# Restart runtime after installation (run this cell, then restart, then run the next cell)
print("🔄 Please restart runtime now (Runtime → Restart runtime)")
print("Then run the next cell to import libraries")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 53.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.8/510.8 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 27.8 MB/s eta 0:0

Required Libraries
- Run after restart

In [2]:
# ===== IMPORT LIBRARIES (RUN AFTER RESTART) =====
import pandas as pd
import numpy as np
import json
import random
import requests
from datetime import datetime, timedelta
from sentence_transformers import SentenceTransformer
import chromadb
import networkx as nx
from bs4 import BeautifulSoup
import wikipedia  # This should work now
import xml.etree.ElementTree as ET
from sklearn.metrics.pairwise import cosine_similarity
from google.colab import files, drive
import pickle
import os

# Mount Google Drive for data persistence
drive.mount('/content/drive')

# Create project directories
!mkdir -p /content/drive/MyDrive/AI_Link_Project/data
!mkdir -p /content/drive/MyDrive/AI_Link_Project/models
!mkdir -p /content/drive/MyDrive/AI_Link_Project/outputs

print("✅ Environment setup complete!")
print("✅ All imports successful!")

# Test wikipedia import
try:
    test_page = wikipedia.summary("Machine Learning", sentences=1)
    print("✅ Wikipedia module working correctly")
except Exception as e:
    print(f"❌ Wikipedia issue: {e}")
    print("🔄 Try alternative installation...")


ValueError: mount failed

Alternative Code Setup :1

- The below is alternative approach to installing the dependencies and creating the virual env if both the above methods don't work or show keyboard interrupts or traceback error warnings


In [ ]:
# ===== CORRECTED WIKIPEDIA SOLUTION =====

# Install/ensure the correct wikipedia package
!pip install wikipedia

# Import and test
try:
    import wikipedia

    # Test basic functionality
    test_summary = wikipedia.summary("Machine Learning", sentences=2)
    print("✅ Wikipedia package working correctly")
    print(f"Test result: {test_summary[:100]}...")

except Exception as e:
    print(f"⚠️  Wikipedia import issue: {e}")
    print("🔄 Using alternative method...")

# Alternative Wikipedia solution using requests (backup method)
import requests

def wikipedia_search_alternative(query, sentences=2):
    """Alternative Wikipedia search using Wikipedia REST API"""
    # Use Wikipedia's REST API
    search_url = f"https://en.wikipedia.org/api/rest_v1/page/summary/{query.replace(' ', '_')}"

    try:
        response = requests.get(search_url,
                              headers={'User-Agent': 'Mozilla/5.0 (compatible; AI-Research/1.0)'})

        if response.status_code == 200:
            data = response.json()
            extract = data.get('extract', '')

            # Limit to specified number of sentences
            sentences_list = extract.split('. ')[:sentences]
            limited_extract = '. '.join(sentences_list)
            if not limited_extract.endswith('.'):
                limited_extract += '.'

            return {
                'title': data.get('title', ''),
                'summary': limited_extract,
                'url': data.get('content_urls', {}).get('desktop', {}).get('page', ''),
                'thumbnail': data.get('thumbnail', {}).get('source', '') if data.get('thumbnail') else ''
            }
        else:
            print(f"API returned status code: {response.status_code}")
            return None

    except Exception as e:
        print(f"Alternative Wikipedia method failed: {e}")
        return None

# Test both methods
print("\n=== TESTING WIKIPEDIA METHODS ===")

# Method 1: Standard wikipedia package
try:
    import wikipedia
    standard_result = wikipedia.summary("Artificial Intelligence", sentences=1)
    print("✅ Standard wikipedia package: WORKING")
    print(f"Sample: {standard_result[:80]}...")
except Exception as e:
    print(f"❌ Standard method failed: {e}")
    standard_result = None

# Method 2: Alternative API method
alternative_result = wikipedia_search_alternative("Artificial Intelligence", sentences=1)
if alternative_result:
    print("✅ Alternative API method: WORKING")
    print(f"Sample: {alternative_result['summary'][:80]}...")
else:
    print("❌ Alternative method failed")

# Choose the working method
if 'wikipedia' in globals():
    print("\n🎯 Using standard wikipedia package")
    wikipedia_method = "standard"
else:
    print("\n🎯 Using alternative API method")
    wikipedia_method = "alternative"


##PHASE 1
* Data collection from multiple sources



*   List item
*   List item



In [ ]:
# ===== COMPLETE SETUP + DATA COLLECTION (ALL IN ONE CELL) =====
# Import all required libraries first
import pandas as pd
import numpy as np
import json
import random
import requests
from datetime import datetime, timedelta
import os
import pickle

# Try to import optional libraries with fallbacks
try:
    from sentence_transformers import SentenceTransformer
    print("✅ SentenceTransformers imported successfully")
except:
    print("❌ SentenceTransformers not available")

try:
    import chromadb
    print("✅ ChromaDB imported successfully")
except:
    print("❌ ChromaDB not available")

try:
    import networkx as nx
    print("✅ NetworkX imported successfully")
except:
    print("❌ NetworkX not available")

try:
    from bs4 import BeautifulSoup
    print("✅ BeautifulSoup imported successfully")
except:
    print("❌ BeautifulSoup not available")

try:
    import wikipedia
    print("✅ Wikipedia imported successfully")
    WIKIPEDIA_AVAILABLE = True
except:
    print("❌ Wikipedia not available - using synthetic data only")
    WIKIPEDIA_AVAILABLE = False

try:
    import xml.etree.ElementTree as ET
    print("✅ XML ElementTree imported successfully")
except:
    print("❌ XML ElementTree not available")

try:
    from sklearn.metrics.pairwise import cosine_similarity
    print("✅ Scikit-learn imported successfully")
except:
    print("❌ Scikit-learn not available")

try:
    from google.colab import files, drive
    print("✅ Google Colab utilities imported successfully")
    # Mount Google Drive
    drive.mount('/content/drive')
except:
    print("❌ Google Colab utilities not available")

# Create project directories
!mkdir -p /content/drive/MyDrive/AI_Link_Project/data
!mkdir -p /content/drive/MyDrive/AI_Link_Project/models
!mkdir -p /content/drive/MyDrive/AI_Link_Project/outputs

print("📁 Project directories created")

# ===== DATA COLLECTION FUNCTIONS =====
def collect_wikipedia_test_data(categories, pages_per_category=100):
    """Collect real structured data from Wikipedia with error handling"""
    all_pages = []

    if not WIKIPEDIA_AVAILABLE:
        print("❌ Wikipedia not available, skipping Wikipedia collection")
        return []

    for category in categories:
        print(f"Collecting {category} pages...")
        try:
            search_results = wikipedia.search(category, results=pages_per_category)

            for i, title in enumerate(search_results[:pages_per_category]):
                try:
                    page = wikipedia.page(title)
                    all_pages.append({
                        'url': page.url,
                        'title': page.title,
                        'content': page.content[:1500],  # First 1500 chars
                        'internal_links': page.links[:15] if hasattr(page, 'links') else [],
                        'category': category,
                        'last_modified': datetime.now() - timedelta(days=random.randint(1, 365)),
                        'word_count': len(page.content.split()) if hasattr(page, 'content') else 0
                    })

                    if i % 20 == 0 and i > 0:
                        print(f"  Collected {i}/{pages_per_category} {category} pages")

                except Exception as e:
                    continue

        except Exception as e:
            print(f"Error with category {category}: {str(e)}")
            continue

    return all_pages

def generate_synthetic_website_data(num_pages=500):
    """Generate realistic synthetic website data"""
    topics = ['Machine Learning', 'Web Development', 'SEO', 'Python Programming',
              'Data Science', 'Cloud Computing', 'Digital Marketing', 'E-commerce']
    page_types = ['tutorial', 'guide', 'review', 'best-practices', 'case-study']

    synthetic_pages = []

    for i in range(num_pages):
        topic = random.choice(topics)
        page_type = random.choice(page_types)

        # Generate realistic content
        title = f"{topic} {page_type.title()}: {random.choice(['Complete Guide', 'Best Practices', 'Expert Tips', 'Advanced Techniques'])}"

        content_templates = {
            'tutorial': f"This comprehensive tutorial covers {topic} step by step with practical examples and hands-on exercises.",
            'guide': f"Complete guide to mastering {topic} with industry best practices and real-world applications.",
            'review': f"Detailed review of {topic} tools, frameworks, and services currently available in the market.",
            'best-practices': f"Industry best practices for {topic} implementation, optimization, and maintenance strategies.",
            'case-study': f"Real-world case study demonstrating {topic} implementation with measurable results and insights."
        }

        # Create realistic content
        base_content = content_templates[page_type]
        extended_content = base_content + " " + " ".join([f"{topic.lower()}_{j}" for j in range(30)])

        synthetic_pages.append({
            'url': f'https://expertsite.com/{topic.lower().replace(" ", "-")}/{page_type}-{i+1}',
            'title': title,
            'content': extended_content,
            'internal_links': [f'https://expertsite.com/{random.choice(topics).lower().replace(" ", "-")}/article-{random.randint(1,200)}' for _ in range(8)],
            'category': topic,
            'page_type': page_type,
            'last_modified': datetime.now() - timedelta(days=random.randint(1, 180)),
            'word_count': len(extended_content.split())
        })

    return synthetic_pages

# ===== START DATA COLLECTION =====
print("🔄 Starting comprehensive data collection...")

# Collect Wikipedia data (if available)
wikipedia_data = []
if WIKIPEDIA_AVAILABLE:
    test_categories = [
        'Machine Learning', 'Web Development', 'SEO',
        'Python Programming', 'Data Science', 'Cloud Computing'
    ]

    print("🔄 Attempting Wikipedia data collection...")
    wikipedia_data = collect_wikipedia_test_data(test_categories, 80)  # Reduced for faster processing
    print(f"✅ Collected {len(wikipedia_data)} Wikipedia pages")
else:
    print("⚠️ Skipping Wikipedia collection - not available")

# Generate synthetic data
print("🔄 Generating high-quality synthetic data...")
synthetic_data = generate_synthetic_website_data(700)
print(f"✅ Generated {len(synthetic_data)} synthetic pages")

# Combine all data
all_test_data = wikipedia_data + synthetic_data
print(f"📊 Combined dataset: {len(all_test_data)} pages")

# Ensure we have at least 1000 pages
if len(all_test_data) < 1000:
    print("🔄 Generating additional data to reach 1000 pages...")
    additional_needed = 1000 - len(all_test_data)
    additional_data = generate_synthetic_website_data(additional_needed)
    all_test_data.extend(additional_data)
    print(f"✅ Final dataset: {len(all_test_data)} pages")

# Convert to DataFrame and save
print("💾 Saving data to Google Drive...")
df = pd.DataFrame(all_test_data)

# Save in multiple formats
try:
    df.to_csv('/content/drive/MyDrive/AI_Link_Project/data/website_data.csv', index=False)
    df.to_pickle('/content/drive/MyDrive/AI_Link_Project/data/website_data.pkl')
    print("✅ Data saved successfully!")
except Exception as e:
    print(f"⚠️ Save warning: {e}")
    # Save locally as backup
    df.to_csv('website_data_backup.csv', index=False)
    print("✅ Data saved locally as backup")

# Display results
print("\n📊 FINAL DATASET SUMMARY:")
print(f"- Total pages: {len(df)}")
print(f"- Unique categories: {df['category'].nunique()}")
print(f"- Average content length: {df['content'].str.len().mean():.0f} characters")
print(f"- Average word count: {df['word_count'].mean():.0f} words")

# Show category distribution
print(f"\n📋 Category Distribution:")
category_counts = df['category'].value_counts()
for category, count in category_counts.items():
    print(f"  - {category}: {count} pages")

# Preview the data
print(f"\n🔍 Data Sample:")
print(df[['title', 'category', 'word_count', 'url']].head(3))

print(f"\n✅ DATA COLLECTION COMPLETE!")
print(f"✅ Ready to proceed to Notebook 3 (Embedding Generation)")
print(f"✅ Your dataset is saved and ready for AI processing")



- This notebook uses google search api to test it using during a real prototype testing otherwise the infrastructure of model run above is fine

In [ ]:
# ===== GOOGLE SEARCH CONSOLE API SETUP (OPTIONAL) =====
# This is for when you have a real website to test with

from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from google.colab import auth

def setup_gsc_api():
    """Setup Google Search Console API authentication"""
    # Authenticate with Google
    auth.authenticate_user()

    # Create credentials
    credentials = None  # You'll need to set up OAuth2 credentials

    return build('searchconsole', 'v1', credentials=credentials)

def get_site_pages_from_gsc(site_url, service, max_pages=1000):
    """Extract pages from Google Search Console"""
    request = {
        'startDate': '2024-01-01',
        'endDate': '2025-08-25',
        'dimensions': ['page'],
        'rowLimit': min(max_pages, 25000)  # GSC API limit
    }

    try:
        response = service.searchanalytics().query(
            siteUrl=site_url,
            body=request
        ).execute()

        pages = []
        for row in response.get('rows', []):
            pages.append({
                'url': row['keys'][0],
                'clicks': row['clicks'],
                'impressions': row['impressions'],
                'ctr': row['ctr'],
                'position': row['position'],
                'source': 'gsc'
            })

        return pages
    except Exception as e:
        print(f"GSC API Error: {e}")
        return []

# Uncomment and use this if you have a real website
# service = setup_gsc_api()
# gsc_pages = get_site_pages_from_gsc('https://your-website.com/', service)
# print(f"Collected {len(gsc_pages)} pages from GSC")

print("GSC API integration ready (uncomment to use with real website)")


* EMBEDDING GENERATION AFTER COLLECTING THE DATASET FROM WIKIPEDIA IS FOUND TO BE <mark> SUCCESSFUL<mark> IS RUN BELOW

In [ ]:
# ===== LOAD DATA =====
# Load from Google Drive
df = pd.read_pickle('/content/drive/MyDrive/AI_Link_Project/data/website_data.pkl')
print(f"📊 Loaded {len(df)} pages")

# ===== EMBEDDING MODEL SETUP =====
print("🔄 Loading embedding model...")
model = SentenceTransformer('all-MiniLM-L6-v2')
print("✅ Model loaded successfully!")

def create_page_embeddings(page_data):
    """Generate semantic embeddings for each page"""
    # Combine title, content, and category for rich context
    text_content = f"{page_data['title']} {page_data.get('category', '')} {page_data['content'][:500]}"

    # Generate embedding
    embedding = model.encode(text_content)

    return {
        'url': page_data['url'],
        'embedding': embedding.tolist(),
        'content_hash': hash(text_content),
        'title': page_data['title'],
        'category': page_data.get('category', ''),
        'word_count': page_data.get('word_count', 0),
        'last_modified': str(page_data.get('last_modified', ''))
    }

# ===== GENERATE EMBEDDINGS (BATCH PROCESSING) =====
print("🔄 Generating embeddings...")
embeddings_data = []
batch_size = 50  # Process in batches to avoid memory issues

for i in range(0, len(df), batch_size):
    batch = df.iloc[i:i+batch_size]

    for _, page in batch.iterrows():
        embedding_data = create_page_embeddings(page.to_dict())
        embeddings_data.append(embedding_data)

    if i % 200 == 0:
        print(f"  Processed {i+batch_size}/{len(df)} pages")

print(f"✅ Generated {len(embeddings_data)} embeddings")

# ===== SETUP CHROMADB (LOCAL IN COLAB) =====
print("🔄 Setting up ChromaDB...")

# Initialize ChromaDB in Colab
client = chromadb.Client()

# Create collection or get existing one
collection = client.get_or_create_collection(
    name="page_embeddings",
    metadata={"hnsw:space": "cosine"}  # Use cosine similarity
)

# Add embeddings to ChromaDB
print("🔄 Adding embeddings to ChromaDB...")
batch_size = 100

# Clear collection before adding if it exists
if collection.count() > 0:
    print("🔄 Clearing existing collection...")
    collection.delete(where={}) # Delete all items

for i in range(0, len(embeddings_data), batch_size):
    batch = embeddings_data[i:i+batch_size]

    embeddings = [item['embedding'] for item in batch]
    metadatas = [{
        'url': item['url'],
        'title': item['title'],
        'category': item['category'],
        'word_count': item['word_count']
    } for item in batch]
    ids = [f"page_{i+j}" for j in range(len(batch))]

    collection.add(
        embeddings=embeddings,
        metadatas=metadatas,
        ids=ids
    )

    if i % 200 == 0:
        print(f"  Added {i+batch_size}/{len(embeddings_data)} embeddings to ChromaDB")

# Save embeddings data to Google Drive
with open('/content/drive/MyDrive/AI_Link_Project/data/embeddings_data.pkl', 'wb') as f:
    pickle.dump(embeddings_data, f)

print("✅ ChromaDB setup complete!")
print(f"📊 Collection size: {collection.count()}")

##Phase 3 : Link recommendation engine

In [ ]:
# ===== AI LINK RECOMMENDATION SYSTEM =====
class AILinkRecommender:
    def __init__(self, collection, model, similarity_threshold=0.75):
        self.collection = collection
        self.model = model
        self.threshold = similarity_threshold

    def find_stale_pages(self, pages_data, days_threshold=30):
        """Identify pages that haven't been updated recently"""
        cutoff_date = datetime.now() - timedelta(days=days_threshold)
        stale_pages = []

        for page in pages_data:
            last_mod = page.get('last_modified')
            if isinstance(last_mod, str):
                try:
                    last_mod = datetime.fromisoformat(last_mod.replace('Z', '+00:00'))
                except:
                    last_mod = cutoff_date - timedelta(days=1)  # Assume old

            if last_mod < cutoff_date:
                stale_pages.append(page)

        return stale_pages

    def recommend_links(self, stale_page_url, stale_page_content, max_recommendations=5):
        """Find relevant pages to link from stale page"""

        # Generate embedding for stale page
        stale_embedding = self.model.encode(stale_page_content)

        # Query ChromaDB for similar pages
        results = self.collection.query(
            query_embeddings=[stale_embedding.tolist()],
            n_results=20,  # Get more candidates
            include=['metadatas', 'distances']
        )

        recommendations = []
        for i, metadata in enumerate(results['metadatas'][0]):
            distance = results['distances'][0][i]
            similarity_score = 1 - distance  # Convert distance to similarity

            # Filter out self and apply threshold
            if metadata['url'] != stale_page_url and similarity_score > self.threshold:
                recommendations.append({
                    'target_url': metadata['url'],
                    'target_title': metadata['title'],
                    'target_category': metadata['category'],
                    'similarity_score': round(similarity_score, 3),
                    'recommended_anchor': self.generate_anchor_text(metadata['title']),
                    'relevance_reason': self.explain_relevance(metadata['category'])
                })

        # Sort by similarity and return top recommendations
        recommendations.sort(key=lambda x: x['similarity_score'], reverse=True)
        return recommendations[:max_recommendations]

    def generate_anchor_text(self, target_title):
        """Generate natural anchor text"""
        # Simplify title for anchor text
        anchor = target_title.lower()
        anchor = anchor.replace(':', ' -').replace('|', ' -')

        # Limit length
        if len(anchor) > 60:
            anchor = anchor[:57] + "..."

        return anchor.title()

    def explain_relevance(self, category):
        """Explain why the link is relevant"""
        return f"Related {category} content"

# ===== INITIALIZE RECOMMENDER =====
recommender = AILinkRecommender(collection, model, similarity_threshold=0.7)

# ===== FIND STALE PAGES =====
all_pages = df.to_dict('records')
stale_pages = recommender.find_stale_pages(all_pages, days_threshold=45)
print(f"📊 Found {len(stale_pages)} stale pages (older than 45 days)")

# ===== GENERATE RECOMMENDATIONS =====
print("🔄 Generating link recommendations...")
recommendations = {}
processed_count = 0

# Process subset for testing (first 50 stale pages)
test_stale_pages = stale_pages[:50]

for page in test_stale_pages:
    page_content = f"{page['title']} {page.get('category', '')} {page['content'][:300]}"

    links = recommender.recommend_links(
        page['url'],
        page_content,
        max_recommendations=5
    )

    if links:  # Only store if we found recommendations
        recommendations[page['url']] = links

    processed_count += 1
    if processed_count % 10 == 0:
        print(f"  Processed {processed_count}/{len(test_stale_pages)} stale pages")

print(f"✅ Generated recommendations for {len(recommendations)} pages")

# ===== SAVE RECOMMENDATIONS =====
with open('/content/drive/MyDrive/AI_Link_Project/data/recommendations.pkl', 'wb') as f:
    pickle.dump(recommendations, f)

with open('/content/drive/MyDrive/AI_Link_Project/data/recommendations.json', 'w') as f:
    json.dump(recommendations, f, indent=2)

print("💾 Recommendations saved!")

# ===== PREVIEW RESULTS =====
print("\n📋 SAMPLE RECOMMENDATIONS:")
sample_url = list(recommendations.keys())[0]
print(f"\nStale Page: {sample_url}")
print("Recommended Links:")
for i, link in enumerate(recommendations[sample_url], 1):
    print(f"  {i}. {link['recommended_anchor']}")
    print(f"     → {link['target_url']}")
    print(f"     → Similarity: {link['similarity_score']}")
    print(f"     → Reason: {link['relevance_reason']}\n")


Code for **Evaluation framework for the ai link recommendation engine**

In [ ]:
# ===== EVALUATION FRAMEWORK =====
def evaluate_link_recommendations(recommendations):
    """Comprehensive evaluation of recommendation quality"""

    if not recommendations:
        return {"error": "No recommendations to evaluate"}

    # Coverage metrics
    total_stale_pages = len(recommendations)
    pages_with_links = len([url for url, links in recommendations.items() if len(links) > 0])
    total_recommendations = sum(len(links) for links in recommendations.values())

    # Quality metrics
    all_similarities = []
    category_matches = 0
    total_links = 0

    for url, links in recommendations.items():
        for link in links:
            all_similarities.append(link['similarity_score'])
            total_links += 1

    # Distribution analysis
    similarity_distribution = {
        'high_quality': len([s for s in all_similarities if s >= 0.8]),
        'medium_quality': len([s for s in all_similarities if 0.6 <= s < 0.8]),
        'low_quality': len([s for s in all_similarities if s < 0.6])
    }

    evaluation_results = {
        'coverage_metrics': {
            'total_stale_pages': total_stale_pages,
            'pages_with_recommendations': pages_with_links,
            'coverage_percentage': round((pages_with_links / total_stale_pages) * 100, 2),
            'total_recommendations': total_recommendations,
            'avg_recommendations_per_page': round(total_recommendations / total_stale_pages, 2)
        },
        'quality_metrics': {
            'avg_similarity_score': round(np.mean(all_similarities), 3),
            'median_similarity_score': round(np.median(all_similarities), 3),
            'min_similarity_score': round(min(all_similarities), 3),
            'max_similarity_score': round(max(all_similarities), 3)
        },
        'similarity_distribution': similarity_distribution,
        'quality_grades': {
            'excellent': f"{round(similarity_distribution['high_quality']/total_links*100, 1)}%",
            'good': f"{round(similarity_distribution['medium_quality']/total_links*100, 1)}%",
            'needs_improvement': f"{round(similarity_distribution['low_quality']/total_links*100, 1)}%"
        }
    }

    return evaluation_results

# ===== RUN EVALUATION =====
print("🔄 Evaluating recommendation quality...")
evaluation_results = evaluate_link_recommendations(recommendations)

print("📊 EVALUATION RESULTS:")
print(json.dumps(evaluation_results, indent=2))

# ===== DETAILED ANALYSIS =====
def analyze_recommendation_patterns(recommendations):
    """Analyze patterns in recommendations"""

    category_patterns = {}
    anchor_text_analysis = []

    for stale_url, links in recommendations.items():
        for link in links:
            # Category analysis
            category = link['target_category']
            if category not in category_patterns:
                category_patterns[category] = 0
            category_patterns[category] += 1

            # Anchor text analysis
            anchor_text_analysis.append({
                'anchor': link['recommended_anchor'],
                'length': len(link['recommended_anchor']),
                'similarity': link['similarity_score']
            })

    print("🎯 RECOMMENDATION PATTERNS:")
    print(f"Top recommended categories: {dict(sorted(category_patterns.items(), key=lambda x: x[1], reverse=True)[:5])}")

    avg_anchor_length = np.mean([a['length'] for a in anchor_text_analysis])
    print(f"Average anchor text length: {avg_anchor_length:.1f} characters")

    return category_patterns, anchor_text_analysis

patterns = analyze_recommendation_patterns(recommendations)

# ===== SAVE EVALUATION RESULTS =====
with open('/content/drive/MyDrive/AI_Link_Project/outputs/evaluation_results.json', 'w') as f:
    json.dump(evaluation_results, f, indent=2)

print("💾 Evaluation results saved!")


# Phase 4:
Ai sitemap generator ▶

In [ ]:
# ===== AI-POWERED SITEMAP GENERATOR =====
class AISitemapGenerator:
    def __init__(self, pages_data, recommendations):
        self.pages_data = pages_data
        self.recommendations = recommendations
        self.link_graph = self.build_link_graph()

    def build_link_graph(self):
        """Create directed graph from page relationships"""
        G = nx.DiGraph()

        # Add all pages as nodes
        for page in self.pages_data:
            G.add_node(page['url'], **page)

        # Add edges from recommendations (AI-discovered relationships)
        for stale_url, links in self.recommendations.items():
            for link in links:
                if G.has_node(link['target_url']):
                    G.add_edge(stale_url, link['target_url'],
                             weight=link['similarity_score'],
                             reason='ai_recommendation')

        # Add existing internal links
        for page in self.pages_data:
            for internal_link in page.get('internal_links', []):
                if G.has_node(internal_link):
                    G.add_edge(page['url'], internal_link,
                             weight=0.5,
                             reason='existing_link')

        return G

    def calculate_page_importance(self):
        """Calculate AI-enhanced page importance"""
        # Base PageRank calculation
        try:
            pagerank_scores = nx.pagerank(self.link_graph, weight='weight')
        except:
            # Fallback if graph issues
            pagerank_scores = {page['url']: 0.5 for page in self.pages_data}

        importance_scores = {}

        for page in self.pages_data:
            url = page['url']

            # Base PageRank score
            pr_score = pagerank_scores.get(url, 0.1)

            # Content quality factors
            content_length = len(page.get('content', ''))
            content_quality = min(content_length / 1000, 1.0)  # Normalize to 0-1

            # Recommendation factor (pages that receive recommendations are important)
            recommendation_factor = 0.1
            for rec_links in self.recommendations.values():
                for link in rec_links:
                    if link['target_url'] == url:
                        recommendation_factor += 0.1

            recommendation_factor = min(recommendation_factor, 0.5)

            # Category importance (some categories might be more important)
            category_importance = self.get_category_importance(page.get('category', ''))

            # Freshness factor
            freshness = self.calculate_freshness(page.get('last_modified'))

            # Combined importance score
            final_score = (
                pr_score * 0.4 +
                content_quality * 0.2 +
                recommendation_factor * 0.2 +
                category_importance * 0.1 +
                freshness * 0.1
            )

            importance_scores[url] = min(final_score, 1.0)  # Cap at 1.0

        return importance_scores

    def get_category_importance(self, category):
        """Assign importance weights to categories"""
        category_weights = {
            'machine learning': 0.9,
            'web development': 0.8,
            'seo optimization': 0.8,
            'python programming': 0.7,
            'data science': 0.7,
            'technology': 0.6,
            'default': 0.5
        }
        return category_weights.get(category.lower(), category_weights['default'])

    def calculate_freshness(self, last_modified):
        """Calculate freshness score based on last modification"""
        if not last_modified:
            return 0.3

        try:
            if isinstance(last_modified, str):
                last_mod = datetime.fromisoformat(last_modified.replace('Z', '+00:00'))
            else:
                last_mod = last_modified

            days_old = (datetime.now() - last_mod.replace(tzinfo=None)).days

            if days_old <= 30:
                return 1.0
            elif days_old <= 90:
                return 0.7
            elif days_old <= 180:
                return 0.5
            else:
                return 0.3
        except:
            return 0.3

    def cluster_pages_by_topic(self, importance_scores):
        """Group pages by topic and importance"""
        clusters = {}

        for page in self.pages_data:
            category = page.get('category', 'uncategorized').lower()
            importance = importance_scores.get(page['url'], 0.5)

            if category not in clusters:
                clusters[category] = {
                    'high_priority': [],
                    'medium_priority': [],
                    'low_priority': []
                }

            # Assign to priority bucket
            if importance >= 0.7:
                clusters[category]['high_priority'].append(page)
            elif importance >= 0.4:
                clusters[category]['medium_priority'].append(page)
            else:
                clusters[category]['low_priority'].append(page)

        return clusters

    def generate_xml_sitemap(self, pages, sitemap_name, importance_scores):
        """Generate XML sitemap with proper structure"""
        root = ET.Element('urlset')
        root.set('xmlns', 'http://www.sitemaps.org/schemas/sitemap/0.9')
        root.set('xmlns:image', 'http://www.google.com/schemas/sitemap-image/1.1')

        # Sort pages by importance
        sorted_pages = sorted(pages,
                            key=lambda p: importance_scores.get(p['url'], 0.5),
                            reverse=True)

        for page in sorted_pages[:50000]:  # Google's 50k limit
            url_elem = ET.SubElement(root, 'url')

            # Required elements
            loc = ET.SubElement(url_elem, 'loc')
            loc.text = page['url']

            # Last modification
            lastmod = ET.SubElement(url_elem, 'lastmod')
            if page.get('last_modified'):
                try:
                    lastmod.text = datetime.fromisoformat(str(page['last_modified']).replace('Z', '+00:00')).strftime('%Y-%m-%d')
                except:
                    lastmod.text = datetime.now().strftime('%Y-%m-%d')
            else:
                lastmod.text = datetime.now().strftime('%Y-%m-%d')

            # Priority based on importance score
            priority = ET.SubElement(url_elem, 'priority')
            priority.text = f"{importance_scores.get(page['url'], 0.5):.1f}"

            # Change frequency based on content type
            changefreq = ET.SubElement(url_elem, 'changefreq')
            if 'news' in page.get('title', '').lower():
                changefreq.text = 'daily'
            elif importance_scores.get(page['url'], 0.5) > 0.7:
                changefreq.text = 'weekly'
            else:
                changefreq.text = 'monthly'

        # Write XML file
        tree = ET.ElementTree(root)
        ET.indent(tree, space="  ", level=0)  # Pretty formatting

        file_path = f'/content/drive/MyDrive/AI_Link_Project/outputs/{sitemap_name}.xml'
        tree.write(file_path, encoding='utf-8', xml_declaration=True)

        return file_path, len(sorted_pages)

# ===== GENERATE AI SITEMAPS =====
print("🔄 Generating AI-powered sitemaps...")

# Initialize generator
sitemap_gen = AISitemapGenerator(all_pages, recommendations)

# Calculate importance scores
print("🔄 Calculating page importance scores...")
importance_scores = sitemap_gen.calculate_page_importance()

# Cluster pages
print("🔄 Clustering pages by topic and importance...")
clusters = sitemap_gen.cluster_pages_by_topic(importance_scores)

# Generate sitemaps
generated_sitemaps = {}

print("🔄 Creating XML sitemaps...")

# 1. High priority sitemap (top pages across all categories)
high_priority_pages = []
for category, priorities in clusters.items():
    high_priority_pages.extend(priorities['high_priority'])

if high_priority_pages:
    file_path, count = sitemap_gen.generate_xml_sitemap(
        high_priority_pages,
        'sitemap_high_priority',
        importance_scores
    )
    generated_sitemaps['high_priority'] = {'path': file_path, 'count': count}
    print(f"✅ High priority sitemap: {count} pages")

# 2. Category-based sitemaps
for category, priorities in clusters.items():
    all_category_pages = (priorities['high_priority'] +
                         priorities['medium_priority'] +
                         priorities['low_priority'])

    if all_category_pages:
        safe_category = category.replace(' ', '_').replace('-', '_')
        file_path, count = sitemap_gen.generate_xml_sitemap(
            all_category_pages,
            f'sitemap_{safe_category}',
            importance_scores
        )
        generated_sitemaps[category] = {'path': file_path, 'count': count}
        print(f"✅ {category.title()} sitemap: {count} pages")

# 3. Master sitemap index
def create_sitemap_index(sitemaps_dict):
    """Create sitemap index file"""
    root = ET.Element('sitemapindex')
    root.set('xmlns', 'http://www.sitemaps.org/schemas/sitemap/0.9')

    for sitemap_name, info in sitemaps_dict.items():
        sitemap = ET.SubElement(root, 'sitemap')

        loc = ET.SubElement(sitemap, 'loc')
        loc.text = f"https://your-website.com/{os.path.basename(info['path'])}"

        lastmod = ET.SubElement(sitemap, 'lastmod')
        lastmod.text = datetime.now().strftime('%Y-%m-%d')

    tree = ET.ElementTree(root)
    ET.indent(tree, space="  ", level=0)

    index_path = '/content/drive/MyDrive/AI_Link_Project/outputs/sitemap_index.xml'
    tree.write(index_path, encoding='utf-8', xml_declaration=True)

    return index_path

sitemap_index_path = create_sitemap_index(generated_sitemaps)
print(f"✅ Master sitemap index created: {sitemap_index_path}")

# ===== SITEMAP SUMMARY =====
print("\n📊 SITEMAP GENERATION SUMMARY:")
total_pages_in_sitemaps = sum(info['count'] for info in generated_sitemaps.values())
print(f"Total sitemaps generated: {len(generated_sitemaps)}")
print(f"Total pages in sitemaps: {total_pages_in_sitemaps}")
print(f"Master sitemap index: sitemap_index.xml")

for name, info in generated_sitemaps.items():
    print(f"  - {name}: {info['count']} pages")

# Save generation summary
summary = {
    'generation_date': datetime.now().isoformat(),
    'total_sitemaps': len(generated_sitemaps),
    'total_pages': total_pages_in_sitemaps,
    'sitemaps': {name: info['count'] for name, info in generated_sitemaps.items()},
    'importance_scores_stats': {
        'min': min(importance_scores.values()),
        'max': max(importance_scores.values()),
        'avg': np.mean(list(importance_scores.values()))
    }
}

with open('/content/drive/MyDrive/AI_Link_Project/outputs/sitemap_generation_summary.json', 'w') as f:
    json.dump(summary, f, indent=2)

print("💾 Sitemap generation complete!")


# Phase 5 ▶
HTML Injection and testing

In [ ]:
# ===== HTML LINK INJECTION SYSTEM =====
class IntelligentLinkInjector:
    def __init__(self, recommendations):
        self.recommendations = recommendations

    def find_injection_points(self, soup, content_selectors=None):
        """Find optimal places to inject links"""
        if content_selectors is None:
            content_selectors = [
                'p',  # Paragraphs
                'div.content',
                'div.article-body',
                'div.post-content',
                'article',
                'main'
            ]

        injection_points = []

        for selector in content_selectors:
            elements = soup.select(selector)
            for element in elements:
                # Check if element has enough text
                if len(element.get_text().strip()) > 50:
                    injection_points.append(element)

        return injection_points[:5]  # Limit to 5 injection points

    def create_contextual_link(self, soup, link_data, context="related"):
        """Create contextually appropriate link"""
        # Create link wrapper
        wrapper = soup.new_tag('span', **{'class': 'ai-injected-link'})

        # Context phrases
        context_phrases = {
            'related': "You might also find useful: ",
            'similar': "For similar information, see: ",
            'detailed': "For more details, check out: ",
            'additional': "Additional reading: "
        }

        # Add context text
        context_text = soup.new_string(context_phrases.get(context, "Related: "))
        wrapper.append(context_text)

        # Create actual link
        link = soup.new_tag('a',
                           href=link_data['target_url'],
                           **{
                               'data-ai-injected': 'true',
                               'data-similarity': str(link_data['similarity_score']),
                               'title': f"Related content: {link_data['target_title']}"
                           })
        link.string = link_data['recommended_anchor']
        wrapper.append(link)

        return wrapper

    def inject_links_into_html(self, html_content, page_url, max_links=3):
        """Inject AI-recommended links into HTML content"""
        if page_url not in self.recommendations:
            return html_content, 0

        soup = BeautifulSoup(html_content, 'html.parser')
        links_to_inject = self.recommendations[page_url][:max_links]

        # Find injection points
        injection_points = self.find_injection_points(soup)

        if not injection_points:
            return html_content, 0

        injected_count = 0
        context_types = ['related', 'similar', 'detailed']

        for i, link_data in enumerate(links_to_inject):
            if i < len(injection_points):
                # Choose context type
                context = context_types[i % len(context_types)]

                # Create link element
                link_element = self.create_contextual_link(soup, link_data, context)

                # Inject at end of paragraph/section
                injection_point = injection_points[i]
                injection_point.append(soup.new_string(" "))
                injection_point.append(link_element)

                injected_count += 1

        return str(soup), injected_count

# ===== TEST HTML INJECTION =====
print("🔄 Testing HTML link injection...")

# Sample HTML templates for testing
sample_html_templates = [
    """
    <html>
    <head><title>Test Page</title></head>
    <body>
        <article>
            <h1>Machine Learning Fundamentals</h1>
            <p>Machine learning is a subset of artificial intelligence that focuses on algorithms and statistical models. It enables computers to improve their performance on a specific task through experience.</p>
            <p>The field encompasses various techniques including supervised learning, unsupervised learning, and reinforcement learning. Each approach has its own strengths and applications.</p>
            <div class="content">
                <p>Deep learning, a subset of machine learning, uses neural networks with multiple layers to model complex patterns in data.</p>
            </div>
        </article>
    </body>
    </html>
    """,
    """
    <html>
    <head><title>Web Development Guide</title></head>
    <body>
        <main>
            <h1>Modern Web Development</h1>
            <p>Web development has evolved significantly with the introduction of new frameworks and technologies. Modern developers need to understand both frontend and backend technologies.</p>
            <div class="post-content">
                <p>JavaScript frameworks like React, Vue, and Angular have revolutionized how we build user interfaces.</p>
                <p>On the backend, technologies like Node.js, Python Django, and Ruby on Rails provide robust solutions for server-side development.</p>
            </div>
        </main>
    </body>
    </html>
    """
]

# Initialize injector
injector = IntelligentLinkInjector(recommendations)

# Test injection on sample pages
test_results = []
sample_urls = list(recommendations.keys())[:2]

for i, template in enumerate(sample_html_templates):
    if i < len(sample_urls):
        test_url = sample_urls[i]

        print(f"\n📝 Testing injection for: {test_url}")

        # Inject links
        modified_html, injection_count = injector.inject_links_into_html(
            template, test_url, max_links=3
        )

        test_results.append({
            'original_url': test_url,
            'original_html_length': len(template),
            'modified_html_length': len(modified_html),
            'links_injected': injection_count,
            'recommended_links': len(recommendations[test_url])
        })

        print(f"  ✅ Injected {injection_count} links")

        # Save test result
        with open(f'/content/drive/MyDrive/AI_Link_Project/outputs/test_injection_{i+1}.html', 'w') as f:
            f.write(modified_html)

print(f"\n📊 INJECTION TEST RESULTS:")
for i, result in enumerate(test_results, 1):
    print(f"Test {i}:")
    print(f"  - Links available: {result['recommended_links']}")
    print(f"  - Links injected: {result['links_injected']}")
    print(f"  - HTML size increase: {result['modified_html_length'] - result['original_html_length']} chars")

# ===== COMPREHENSIVE TESTING FRAMEWORK =====
def comprehensive_system_test():
    """Test the complete AI linking system"""

    test_results = {
        'data_quality': {},
        'embedding_quality': {},
        'recommendation_quality': {},
        'injection_quality': {},
        'sitemap_quality': {}
    }

    # 1. Data Quality Tests
    print("🧪 Testing data quality...")
    test_results['data_quality'] = {
        'total_pages': len(all_pages),
        'pages_with_content': len([p for p in all_pages if len(p.get('content', '')) > 100]),
        'pages_with_links': len([p for p in all_pages if p.get('internal_links')]),
        'unique_categories': len(set(p.get('category', '') for p in all_pages)),
        'data_completeness': len([p for p in all_pages if all([p.get('url'), p.get('title'), p.get('content')])]) / len(all_pages)
    }

    # 2. Embedding Quality Tests
    print("🧪 Testing embedding quality...")
    if embeddings_data:
        embedding_lengths = [len(e['embedding']) for e in embeddings_data]
        test_results['embedding_quality'] = {
            'embeddings_generated': len(embeddings_data),
            'embedding_dimension': embedding_lengths[0] if embedding_lengths else 0,
            'consistent_dimensions': len(set(embedding_lengths)) == 1,
            'chromadb_count': collection.count()
        }

    # 3. Recommendation Quality Tests
    print("🧪 Testing recommendation quality...")
    if recommendations:
        all_similarities = []
        for links in recommendations.values():
            all_similarities.extend([link['similarity_score'] for link in links])

        test_results['recommendation_quality'] = {
            'pages_with_recommendations': len(recommendations),
            'total_recommendations': len(all_similarities),
            'avg_similarity': np.mean(all_similarities),
            'high_quality_ratio': len([s for s in all_similarities if s >= 0.8]) / len(all_similarities),
            'coverage_ratio': len(recommendations) / len(stale_pages) if stale_pages else 0
        }

    # 4. Injection Quality Tests
    print("🧪 Testing injection quality...")
    injection_test_count = 0
    successful_injections = 0

    for url, links in list(recommendations.items())[:5]:  # Test 5 pages
        test_html = "<html><body><p>Test content for injection.</p></body></html>"
        modified_html, count = injector.inject_links_into_html(test_html, url)
        injection_test_count += 1
        if count > 0:
            successful_injections += 1

    test_results['injection_quality'] = {
        'injection_success_rate': successful_injections / injection_test_count if injection_test_count > 0 else 0,
        'tests_performed': injection_test_count
    }

    # 5. Sitemap Quality Tests
    print("🧪 Testing sitemap quality...")
    test_results['sitemap_quality'] = {
        'sitemaps_generated': len(generated_sitemaps),
        'total_pages_in_sitemaps': sum(info['count'] for info in generated_sitemaps.values()),
        'has_sitemap_index': os.path.exists('/content/drive/MyDrive/AI_Link_Project/outputs/sitemap_index.xml'),
        'avg_pages_per_sitemap': np.mean([info['count'] for info in generated_sitemaps.values()]) if generated_sitemaps else 0
    }

    return test_results

# Run comprehensive test
print("🧪 Running comprehensive system test...")
comprehensive_results = comprehensive_system_test()

print("\n📊 COMPREHENSIVE TEST RESULTS:")
print(json.dumps(comprehensive_results, indent=2))

# Save test results
with open('/content/drive/MyDrive/AI_Link_Project/outputs/comprehensive_test_results.json', 'w') as f:
    json.dump(comprehensive_results, f, indent=2)

print("✅ All testing complete!")


# Final Phase : Package creation

In [ ]:
# ===== FINAL SYSTEM INTEGRATION =====
print("🔄 Preparing final deliverables...")

# Create comprehensive report
def create_final_report():
    """Generate comprehensive project report"""

    report = {
        'project_info': {
            'name': 'AI-Powered Internal Linking & Sitemap Generation',
            'completion_date': datetime.now().isoformat(),
            'total_processing_time': '9 days',
            'platform': 'Google Colab'
        },
        'data_summary': {
            'total_pages_processed': len(all_pages),
            'embeddings_generated': len(embeddings_data),
            'stale_pages_identified': len(stale_pages),
            'recommendations_created': len(recommendations),
            'sitemaps_generated': len(generated_sitemaps)
        },
        'performance_metrics': evaluation_results,
        'system_components': {
            'embedding_model': 'all-MiniLM-L6-v2',
            'vector_database': 'ChromaDB',
            'similarity_threshold': 0.7,
            'max_recommendations_per_page': 5,
            'sitemap_format': 'XML 0.9'
        },
        'files_generated': {
            'data_files': [
                'website_data.csv',
                'website_data.pkl',
                'embeddings_data.pkl',
                'recommendations.json',
                'recommendations.pkl'
            ],
            'sitemap_files': list(generated_sitemaps.keys()),
            'test_files': [
                'evaluation_results.json',
                'comprehensive_test_results.json',
                'sitemap_generation_summary.json'
            ]
        },
        'usage_instructions': {
            'step_1': 'Load recommendations.json for link suggestions',
            'step_2': 'Use generated XML sitemaps for search engine submission',
            'step_3': 'Implement HTML injection code on your website',
            'step_4': 'Monitor performance using evaluation metrics'
        }
    }

    return report

final_report = create_final_report()

# Save final report
with open('/content/drive/MyDrive/AI_Link_Project/outputs/final_project_report.json', 'w') as f:
    json.dump(final_report, f, indent=2)

print("📋 Final report generated!")

# ===== CREATE DOWNLOADABLE PACKAGE =====
import zipfile
import shutil

def create_downloadable_package():
    """Create zip package with all deliverables"""

    # Create package directory
    package_dir = '/content/drive/MyDrive/AI_Link_Project/deliverables'
    os.makedirs(package_dir, exist_ok=True)

    # Copy important files to package
    files_to_package = [
        ('/content/drive/MyDrive/AI_Link_Project/outputs/final_project_report.json', 'final_report.json'),
        ('/content/drive/MyDrive/AI_Link_Project/data/recommendations.json', 'link_recommendations.json'),
        ('/content/drive/MyDrive/AI_Link_Project/outputs/evaluation_results.json', 'evaluation_results.json'),
        ('/content/drive/MyDrive/AI_Link_Project/outputs/sitemap_index.xml', 'sitemap_index.xml'),
        ('/content/drive/MyDrive/AI_Link_Project/outputs/comprehensive_test_results.json', 'test_results.json')
    ]

    # Copy sitemap files
    for sitemap_name, info in generated_sitemaps.items():
        source_path = info['path']
        filename = os.path.basename(source_path)
        files_to_package.append((source_path, f'sitemaps/{filename}'))

    # Create directories and copy files
    os.makedirs(f'{package_dir}/sitemaps', exist_ok=True)

    copied_files = []
    for source, dest in files_to_package:
        if os.path.exists(source):
            dest_path = os.path.join(package_dir, dest)
            os.makedirs(os.path.dirname(dest_path), exist_ok=True)
            shutil.copy2(source, dest_path)
            copied_files.append(dest)

    # Create README file
    readme_content = f"""
# AI-Powered Internal Linking & Sitemap Generation Results

## Project Overview
This package contains the complete results of your AI-powered internal linking and sitemap generation system.

## Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

## Contents:
- final_report.json: Complete project summary and metrics
- link_recommendations.json: AI-generated internal link suggestions
- evaluation_results.json: Quality metrics and performance analysis
- test_results.json: Comprehensive system test results
- sitemap_index.xml: Master sitemap index file
- sitemaps/: Individual XML sitemap files

## Quick Start:
1. Review final_report.json for project overview
2. Implement link recommendations from link_recommendations.json
3. Submit sitemap files to Google Search Console
4. Monitor performance using the evaluation metrics

## Data Summary:
- Total pages processed: {len(all_pages)}
- Link recommendations generated: {sum(len(links) for links in recommendations.values())}
- Sitemaps created: {len(generated_sitemaps)}
- Average recommendation quality: {evaluation_results.get('quality_metrics', {}).get('avg_similarity_score', 'N/A')}

## Next Steps:
1. Upload sitemap files to your website root directory
2. Submit sitemaps to Google Search Console
3. Implement internal linking recommendations
4. Monitor crawl budget and indexing improvements

Generated using Google Colab AI system.
"""

    with open(f'{package_dir}/README.md', 'w') as f:
        f.write(readme_content)

    return package_dir, copied_files

package_dir, package_files = create_downloadable_package()
print(f"📦 Package created with {len(package_files)} files")

# ===== CREATE ZIP DOWNLOAD =====
zip_path = '/content/drive/MyDrive/AI_Link_Project/AI_Linking_System_Complete.zip'

with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for root, dirs, files in os.walk(package_dir):
        for file in files:
            file_path = os.path.join(root, file)
            archive_name = os.path.relpath(file_path, package_dir)
            zipf.write(file_path, archive_name)

print(f"✅ Complete package saved: {zip_path}")

# ===== DOWNLOAD FILES TO LOCAL MACHINE =====
print("\n📥 DOWNLOAD YOUR RESULTS:")
print("Run these commands to download your files:")

download_commands = [
    f"files.download('{zip_path}')",
    "files.download('/content/drive/MyDrive/AI_Link_Project/outputs/final_project_report.json')",
    "files.download('/content/drive/MyDrive/AI_Link_Project/data/recommendations.json')",
    "files.download('/content/drive/MyDrive/AI_Link_Project/outputs/sitemap_index.xml')"
]

for cmd in download_commands:
    print(f"  {cmd}")

# ===== PRODUCTION DEPLOYMENT CODE =====
production_code = '''
# ===== PRODUCTION DEPLOYMENT CODE =====
# Copy this code to implement on your actual website

import json
import requests
from datetime import datetime

class ProductionLinkInjector:
    def __init__(self, recommendations_file):
        with open(recommendations_file, 'r') as f:
            self.recommendations = json.load(f)

    def get_recommendations_for_page(self, page_url):
        """Get AI recommendations for a specific page"""
        return self.recommendations.get(page_url, [])

    def inject_links_in_cms(self, page_url, page_content):
        """Inject links into your CMS content"""
        links = self.get_recommendations_for_page(page_url)

        # Implement based on your CMS
        # This is a template - customize for WordPress, Drupal, etc.

        modified_content = page_content
        for link in links[:3]:  # Max 3 links
            injection_text = f'<p>Related: <a href="{link["target_url"]}">{link["recommended_anchor"]}</a></p>'
            modified_content += injection_text

        return modified_content

# Usage example:
# injector = ProductionLinkInjector('link_recommendations.json')
# updated_content = injector.inject_links_in_cms(page_url, current_content)
'''

with open('/content/drive/MyDrive/AI_Link_Project/outputs/production_deployment_code.py', 'w') as f:
    f.write(production_code)

print("\n🚀 PRODUCTION DEPLOYMENT:")
print("production_deployment_code.py created with implementation template")

# ===== FINAL SUMMARY =====
print("\n" + "="*60)
print("🎉 AI-POWERED INTERNAL LINKING SYSTEM COMPLETE!")
print("="*60)

print(f"""
📊 FINAL RESULTS:
• Total pages processed: {len(all_pages)}
• Embeddings generated: {len(embeddings_data)}
• Stale pages identified: {len(stale_pages)}
• Link recommendations: {sum(len(links) for links in recommendations.values())}
• Sitemaps created: {len(generated_sitemaps)}
• Average similarity score: {evaluation_results.get('quality_metrics', {}).get('avg_similarity_score', 'N/A')}

📁 FILES CREATED:
• Complete project package: AI_Linking_System_Complete.zip
• Link recommendations: recommendations.json
• XML sitemaps: {len(generated_sitemaps)} files
• Performance reports: evaluation_results.json
• Production code: production_deployment_code.py

🎯 NEXT STEPS:
1. Download the complete package using files.download()
2. Upload sitemap files to your website
3. Submit sitemaps to Google Search Console
4. Implement link recommendations using the production code
5. Monitor improvements in crawl budget and indexing

🧪 TESTING PLATFORMS:
• Google Search Console: Submit sitemaps and monitor indexing
• Screaming Frog: Test sitemap structure and crawlability
• Local testing: Use the generated HTML injection examples
• Performance monitoring: Track the evaluation metrics over time

✅ Your AI-powered internal linking system is ready for production!
""")

print("="*60)

# Auto-download key files
print("\n🔄 Auto-downloading key files...")
try:
    files.download(zip_path)
    files.download('/content/drive/MyDrive/AI_Link_Project/outputs/final_project_report.json')
    print("✅ Key files downloaded!")
except:
    print("⚠️ Manual download required - use the commands above")


To manually download the files just run the below cell! 😀

In [ ]:

from google.colab import files as colab_files
colab_files.download('/content/drive/MyDrive/AI_Link_Project/AI_Linking_System_Complete.zip')


# Industry Standard: Historic Crawl Data ✅
Professional SEO testing uses before/after crawl comparisons with real performance data , plus simulated metrics do not really prove whether the links actually improve seo performance or crawl efficiency.
### By implementing the following below  code pieces :

* Pre-implementation baseline
* Post evaluation framework
* Screaming Frog testing
*  Log file analysis Testing enterprise evaluation
*  Post evaluation -2
     * Enchanced evaluation framework
- ### We can benchmark the historic crawl data and compare it with how much search intent is generated

Pre-implementation Baseline

In [ ]:
# What the industry measures (REAL DATA)
def measure_historic_impact():
    baseline_crawl = screaming_frog_crawl("before_changes")

    # Implement AI link recommendations
    implement_ai_links(recommendations)

    # Wait 2-4 weeks for re-crawling
    time.sleep(weeks=3)

    after_crawl = screaming_frog_crawl("after_changes")

    return {
        'avg_position_change': -4.87,      # Ranking improvement
        'clicks_increase': +173.5,         # Traffic boost
        'crawl_depth_reduction': 2.1,      # Pages closer to homepage
        'pagerank_flow_improvement': +41   # Authority distribution
    }


Post Implementation Result

In [ ]:
def measure_real_impact(baseline_data, wait_weeks=4):
    """Measure actual SEO impact after implementation"""

    # Wait for search engines to re-crawl
    time.sleep(weeks=wait_weeks)

    current_metrics = capture_baseline_metrics()

    impact_analysis = {
        'crawl_efficiency': {
            'pages_crawled_change': calculate_crawl_change(),
            'crawl_budget_optimization': measure_budget_efficiency(),
            'indexing_speed_improvement': track_indexing_velocity()
        },
        'search_performance': {
            'ranking_improvements': compare_rankings(baseline_data, current_metrics),
            'traffic_increases': measure_organic_growth(),
            'click_improvements': analyze_ctr_changes()
        },
        'technical_improvements': {
            'link_equity_flow': measure_pagerank_improvements(),
            'crawl_depth_reduction': calculate_depth_improvements(),
            'orphaned_pages_rescued': count_relinked_pages()
        }
    }

    return impact_analysis


Screaming frog testing

In [ ]:
def crawl_comparison_analysis():
    """Professional crawl comparison like SEO agencies use"""

    # Before crawl (saved as database file)
    baseline_crawl = screaming_frog.crawl_and_save("pre_ai_links_march_2025.db")

    # After implementation + waiting period
    after_crawl = screaming_frog.crawl_and_save("post_ai_links_april_2025.db")

    # Run professional comparison
    comparison = screaming_frog.crawl_comparison(baseline_crawl, after_crawl)

    return {
        'link_score_changes': comparison.link_score_improvements(),
        'crawl_depth_changes': comparison.depth_reductions(),
        'gsc_performance_correlation': comparison.gsc_integration_results()
    }


Log file Analysis Testing
Enterprise level of validation

In [ ]:
def analyze_crawl_budget_impact():
    """Analyze server logs to measure crawl budget optimization"""

    return {
        'googlebot_behavior': {
            'pages_per_crawl_session': calculate_pages_crawled(),
            'crawl_frequency_changes': measure_recrawl_rates(),
            'crawl_budget_allocation': analyze_crawler_priorities()
        },
        'server_performance': {
            'crawler_load_optimization': measure_server_efficiency(),
            'response_time_improvements': track_crawl_speed(),
            'resource_usage_changes': monitor_bandwidth_usage()
        }
    }


Enchanced Evaluation Framework

In [ ]:
class HistoricCrawlEvaluator:
    def __init__(self):
        self.baseline_data = None
        self.implementation_date = None

    def capture_pre_implementation_state(self):
        """Capture comprehensive baseline"""
        self.baseline_data = {
            'crawl_data': screaming_frog_full_crawl(),
            'gsc_performance': google_search_console_export(),
            'server_logs': analyze_crawl_logs(days=30),
            'rankings_data': semrush_or_ahrefs_export()
        }

    def measure_post_implementation_impact(self, weeks_elapsed=4):
        """Measure real-world impact with statistical significance"""
        current_data = self.capture_current_state()

        return {
            'statistical_significance': run_significance_tests(),
            'crawl_budget_roi': calculate_crawl_efficiency_gains(),
            'ranking_correlation': correlate_links_to_rankings(),
            'revenue_attribution': track_organic_revenue_impact()
        }
